In [76]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean
import datetime as dt
import yfinance as yf
import pandas_market_calendars as market_calendar
import datetime as dt
import yfinance as yf
from pyalgotrade import strategy as pyalgo_strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma
from pyalgotrade import plotter
from pyalgotrade.stratanalyzer import returns,drawdown,trades,sharpe
from pyalgotrade import strategy

In [83]:
# (Loads tickers, builds comma-separated groups, sets parameters,
#  pulls Yahoo data & basic fundamentals, computes EMA/ATR/CumRet/MaxDD,
#  and assembles rv_dataframe in one pass.)

stocks = pd.read_csv('sp_500_50_stocks.csv')               # Read tickers from CSV (expects a 'Ticker' column)
end = dt.date.today().isoformat()                          # Use today's date as the data 'end' (YYYY-MM-DD)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""        # Utility: split a list into fixed-size chunks
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups=list(chunks(stocks['Ticker'],100))           # Split the tickers into groups of up to 100
symbol_strings=[]                                          # Will store each group as a single comma-separated string

for i in range (0,len(symbol_groups)):                     # For each group of tickers...
     symbol_strings.append(','.join(symbol_groups[i]))     # ...join tickers with commas and store the string
     # print(symbol_strings[i])                            # (Optional) debug print for the i-th group

# Initialize the short and long windows
EMA_window1 = 21                                           # Fast EMA window (not used later here but kept for context)
EMA_window2 = 55                                           # Medium EMA window (not used later here but kept for context)
EMA_window3 = 200                                          # Long EMA window used for EMA200 calculation
Volume_window = 14                                         # Window for rolling average volume
portfolio_size=10000                                       # Placeholder portfolio size (not used in this block)

        
# use existing variables if they already exist
try:
    end                                                    # If 'end' already exists, keep it
except NameError:
    end = dt.date.today().isoformat()                      # Otherwise set to today

try:
    EMA_window3                                            # Keep existing EMA_window3 if defined
except NameError:
    EMA_window3 = 200                                      # Else default to 200

try:
    Volume_window                                          # Keep existing Volume_window if defined
except NameError:
    Volume_window = 14                                     # Else default to 14

# Column order + dtypes
rv_dtypes = {                                              # Target schema and dtypes for the result table
    'Ticker': 'string',
    'Price': 'float64',
    'Price-to-Earnings Ratio': 'float64',
    'PE Percentile': 'float64',
    'Price-to-Book Ratio': 'float64',
    'PB Percentile': 'float64',
    'Price-to-Sales Ratio': 'float64',
    'PS Percentile': 'float64',
    'EV/EBITDA': 'float64',
    'EV/EBITDA Percentile': 'float64',
    'EV/GP': 'float64',
    'EV/GP Percentile': 'float64',
    'RV Score': 'float64',
    'EMA200': 'float64',
    'AVG_VOL': 'float64',
    'ATR': 'float64',
    'Cumulative Return': 'float64',
    'Max Drawdown': 'float64',
}
rv_columns = list(rv_dtypes.keys())                        # Preserve explicit column order

rows = []  # collect per-row dicts here                     # We'll build a list of dicts, then create the DataFrame once

for symbol_string in symbol_strings:                       # Loop through each comma-separated group of tickers
    # Yahoo fundamentals
    syms = [s.strip().upper() for s in symbol_string.split(",") if s.strip()]  # Normalize tickers (trim & uppercase)
    data = {}                                              # Cache fundamentals for this group to reuse in inner loop
    for s in syms:
        tkr = yf.Ticker(s.replace(".", "-"))               # yfinance uses '-' for class shares (e.g., BRK-B)
        try:
            info = tkr.get_info()                          # Pull Yahoo 'info' dict (can be partial/missing)
        except Exception:
            info = {}                                      # On error, use empty dict to avoid KeyErrors
        data[s] = {                                        # Store only fields needed later in a consistent layout
            "quote": {"peRatio": info.get("trailingPE")},
            "advanced-stats": {
                "priceToBook":     info.get("priceToBook"),
                "priceToSales":    info.get("priceToSalesTrailing12Months"),
                "enterpriseValue": info.get("enterpriseValue"),
                "EBITDA":          info.get("ebitda"),
                "grossProfit":     info.get("grossProfits"),
            }
        }

    for symbol in symbol_string.split(','):                # Iterate actual tickers in this group
        symbol = symbol.strip().upper()                    # Normalize current ticker

        enterprise_value = data.get(symbol, {}).get('advanced-stats', {}).get('enterpriseValue')  # EV
        ebitda          = data.get(symbol, {}).get('advanced-stats', {}).get('EBITDA')            # EBITDA
        gross_profit    = data.get(symbol, {}).get('advanced-stats', {}).get('grossProfit')       # Gross Profit

        try:
            ev_to_ebitda = enterprise_value / ebitda       # Compute EV/EBITDA when both are valid
        except Exception:
            ev_to_ebitda = np.nan                          # Otherwise set as NaN

        try:
            ev_to_gross_profit = enterprise_value / gross_profit  # Compute EV/GP
        except Exception:
            ev_to_gross_profit = np.nan                    # Fallback to NaN on failure

        # Download OHLCV
        try:
            DF_Ind = yf.download(                          # Pull daily OHLCV history from Yahoo
                symbol.replace(".", "-"),
                start="2020-10-01",
                end=end,
                auto_adjust=False,
                progress=False
            )
        except Exception:
            DF_Ind = pd.DataFrame()                        # If download fails, work with an empty frame

        # EMA200
        try:
            DF_Ind['EMA200'] = DF_Ind['Close'].ewm(span=EMA_window3, adjust=False).mean()  # 200-day EMA on Close
            EMA200 = float(DF_Ind["EMA200"].iloc[-1])      # Latest EMA value as a scalar
        except Exception:
            EMA200 = np.nan                                # NaN if missing/insufficient data

        # AVG_VOL (14)
        try:
            DF_Ind['AVG_Vol14'] = DF_Ind['Volume'].rolling(Volume_window).mean()  # 14-day avg volume
            AVG_VOL = float(DF_Ind["AVG_Vol14"].iloc[-1]) / 1_000_000.0           # Convert to millions
        except Exception:
            AVG_VOL = np.nan                                # NaN if data missing

        # Price (last valid Close, with fallback)
        try:
            s_close = pd.to_numeric(DF_Ind['Close'], errors='coerce').dropna()    # Clean Close series
            if not s_close.empty:
                Close = float(s_close.iloc[-1])                                   # Latest valid close
            else:
                raise ValueError("Empty Close series")                            # Trigger fallback
        except Exception:
            try:
                h = yf.Ticker(symbol.replace(".", "-")).history(period="5d", auto_adjust=False)   # Short fallback
                Close = float(pd.to_numeric(h['Close'], errors='coerce').dropna().iloc[-1])       # Use last close
            except Exception:
                Close = np.nan                                                    # If all fails, set NaN

        # ATR (14)
        try:
            high_low   = DF_Ind['High'] - DF_Ind['Low']                           # Range: High-Low
            high_close = (DF_Ind['High'] - DF_Ind['Close'].shift()).abs()         # |High - prev Close|
            low_close  = (DF_Ind['Low']  - DF_Ind['Close'].shift()).abs()         # |Low  - prev Close|
            ranges = pd.concat([high_low, high_close, low_close], axis=1)         # Combine three TR components
            true_range = ranges.max(axis=1)                                       # True Range = max of components
            DF_Ind['ATR'] = true_range.rolling(14).sum() / 14.0                   # 14-day ATR
            ATR = float(DF_Ind["ATR"].iloc[-1])                                   # Latest ATR value
        except Exception:
            ATR = np.nan                                                          # NaN if calculation fails

        # Cumulative Return & Max Drawdown
        try:
            px = DF_Ind['Adj Close'] if 'Adj Close' in DF_Ind else DF_Ind['Close']  # Prefer adjusted close
            px = pd.to_numeric(px, errors='coerce').dropna()                         # Clean series
            if len(px) >= 2:
                cum_return_pct = float((px.iloc[-1] / px.iloc[0] - 1.0) * 100.0)     # Total return (%)
                running_max = px.cummax()                                            # Running peak
                drawdown = (px / running_max - 1.0) * 100.0                          # Drawdown series (%)
                max_drawdown_pct = float(drawdown.min())                              # Most negative drawdown
            else:
                cum_return_pct = np.nan                                              # Not enough data
                max_drawdown_pct = np.nan
        except Exception:
            cum_return_pct = np.nan                                                  # Fallback on any error
            max_drawdown_pct = np.nan

        # Add to rows list (include all columns; use np.nan for placeholders)
        rows.append({
            'Ticker': str(symbol),                                                   # Ticker symbol
            'Price': Close,                                                          # Last close (or fallback)
            'Price-to-Earnings Ratio': data.get(symbol, {}).get('quote', {}).get('peRatio', np.nan),
            'PE Percentile': np.nan,                                                 # Placeholder for later rank
            'Price-to-Book Ratio': data.get(symbol, {}).get('advanced-stats', {}).get('priceToBook', np.nan),
            'PB Percentile': np.nan,                                                 # Placeholder for later rank
            'Price-to-Sales Ratio': data.get(symbol, {}).get('advanced-stats', {}).get('priceToSales', np.nan),
            'PS Percentile': np.nan,                                                 # Placeholder for later rank
            'EV/EBITDA': ev_to_ebitda,                                               # Computed above
            'EV/EBITDA Percentile': np.nan,                                          # Placeholder for later rank
            'EV/GP': ev_to_gross_profit,                                             # Computed above
            'EV/GP Percentile': np.nan,                                              # Placeholder for later rank
            'RV Score': np.nan,                                                      # Placeholder composite score
            'EMA200': EMA200,                                                        # Latest 200-day EMA
            'AVG_VOL': AVG_VOL,                                                      # 14-day avg volume (M)
            'ATR': ATR,                                                              # 14-day ATR
            'Cumulative Return': cum_return_pct,                                      # From first to last point
            'Max Drawdown': max_drawdown_pct,                                        # Worst drawdown in series
        })

# Build the DataFrame once (no per-row concat → no FutureWarning)
rv_dataframe = pd.DataFrame(rows, columns=rv_columns)       # Assemble all rows with a fixed column order

# Enforce dtypes
rv_dataframe = rv_dataframe.astype(rv_dtypes)               # Cast columns to the defined dtypes

rv_dataframe                                                   # Display the final result


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,NaN,18.596970,NaN,3.242715,NaN,15.713103,NaN,9.003108,NaN,NaN,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,NaN,5.171563,NaN,2.518687,NaN,25.644411,NaN,13.604165,NaN,NaN,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,NaN,4.567343,NaN,5.357355,NaN,20.821846,NaN,9.822580,NaN,NaN,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,NaN,-2237.884500,NaN,7.048910,NaN,16.801678,NaN,11.350715,NaN,NaN,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,NaN,5.990781,NaN,7.200601,NaN,24.179172,NaN,11.794173,NaN,NaN,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,NaN,4.828724,NaN,2.161998,NaN,11.632723,NaN,6.948946,NaN,NaN,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,NaN,7.971851,NaN,7.771416,NaN,20.418826,NaN,13.136821,NaN,NaN,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,NaN,1.314494,NaN,0.356535,NaN,15.236817,NaN,7.446478,NaN,NaN,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,NaN,12.070547,NaN,6.108353,NaN,16.132555,NaN,7.073559,NaN,NaN,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,NaN,NaN,1.347060,NaN,0.339561,NaN,22.086128,NaN,1.472225,NaN,NaN,52.102317,1.745207,2.434286,NaN,NaN



**Dealing With Missing Data in Our DataFrame**

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

We can use pandas' isnull method to identify missing data:


In [85]:
# show the missing data lines

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,NaN,18.596970,NaN,3.242715,NaN,15.713103,NaN,9.003108,NaN,NaN,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,NaN,5.171563,NaN,2.518687,NaN,25.644411,NaN,13.604165,NaN,NaN,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,NaN,4.567343,NaN,5.357355,NaN,20.821846,NaN,9.822580,NaN,NaN,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,NaN,-2237.884500,NaN,7.048910,NaN,16.801678,NaN,11.350715,NaN,NaN,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,NaN,5.990781,NaN,7.200601,NaN,24.179172,NaN,11.794173,NaN,NaN,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,NaN,4.828724,NaN,2.161998,NaN,11.632723,NaN,6.948946,NaN,NaN,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,NaN,7.971851,NaN,7.771416,NaN,20.418826,NaN,13.136821,NaN,NaN,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,NaN,1.314494,NaN,0.356535,NaN,15.236817,NaN,7.446478,NaN,NaN,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,NaN,12.070547,NaN,6.108353,NaN,16.132555,NaN,7.073559,NaN,NaN,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,NaN,NaN,1.347060,NaN,0.339561,NaN,22.086128,NaN,1.472225,NaN,NaN,52.102317,1.745207,2.434286,NaN,NaN


In [86]:
# replacing missing data with the average non-NaN data point from that column.

cols = ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']

# make sure they’re numeric
rv_dataframe[cols] = rv_dataframe[cols].apply(pd.to_numeric, errors='coerce')

for col in cols:
    rv_dataframe[col] = rv_dataframe[col].fillna(rv_dataframe[col].mean())
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,NaN,18.596970,NaN,3.242715,NaN,15.713103,NaN,9.003108,NaN,NaN,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,NaN,5.171563,NaN,2.518687,NaN,25.644411,NaN,13.604165,NaN,NaN,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,NaN,4.567343,NaN,5.357355,NaN,20.821846,NaN,9.822580,NaN,NaN,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,NaN,-2237.884500,NaN,7.048910,NaN,16.801678,NaN,11.350715,NaN,NaN,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,NaN,5.990781,NaN,7.200601,NaN,24.179172,NaN,11.794173,NaN,NaN,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,NaN,4.828724,NaN,2.161998,NaN,11.632723,NaN,6.948946,NaN,NaN,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,NaN,7.971851,NaN,7.771416,NaN,20.418826,NaN,13.136821,NaN,NaN,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,NaN,1.314494,NaN,0.356535,NaN,15.236817,NaN,7.446478,NaN,NaN,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,NaN,12.070547,NaN,6.108353,NaN,16.132555,NaN,7.073559,NaN,NaN,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,31.362177,NaN,1.347060,NaN,0.339561,NaN,22.086128,NaN,1.472225,NaN,NaN,52.102317,1.745207,2.434286,NaN,NaN




Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned

In [87]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)] 

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,NaN,18.596970,NaN,3.242715,NaN,15.713103,NaN,9.003108,NaN,NaN,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,NaN,5.171563,NaN,2.518687,NaN,25.644411,NaN,13.604165,NaN,NaN,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,NaN,4.567343,NaN,5.357355,NaN,20.821846,NaN,9.822580,NaN,NaN,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,NaN,-2237.884500,NaN,7.048910,NaN,16.801678,NaN,11.350715,NaN,NaN,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,NaN,5.990781,NaN,7.200601,NaN,24.179172,NaN,11.794173,NaN,NaN,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,NaN,4.828724,NaN,2.161998,NaN,11.632723,NaN,6.948946,NaN,NaN,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,NaN,7.971851,NaN,7.771416,NaN,20.418826,NaN,13.136821,NaN,NaN,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,NaN,1.314494,NaN,0.356535,NaN,15.236817,NaN,7.446478,NaN,NaN,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,NaN,12.070547,NaN,6.108353,NaN,16.132555,NaN,7.073559,NaN,NaN,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,31.362177,NaN,1.347060,NaN,0.339561,NaN,22.086128,NaN,1.472225,NaN,NaN,52.102317,1.745207,2.434286,NaN,NaN


Calculating percentile scores for the following metrics for every stock:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    EV/EBITDA
    EV/GP

In [88]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
  
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,0.285714,18.596970,0.938776,3.242715,0.387755,15.713103,0.387755,9.003108,0.367347,NaN,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,0.224490,5.171563,0.591837,2.518687,0.326531,25.644411,0.857143,13.604165,0.714286,NaN,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,0.163265,4.567343,0.551020,5.357355,0.632653,20.821846,0.775510,9.822580,0.428571,NaN,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,0.979592,-2237.884500,0.020408,7.048910,0.775510,16.801678,0.469388,11.350715,0.510204,NaN,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,0.693878,5.990781,0.653061,7.200601,0.795918,24.179172,0.836735,11.794173,0.571429,NaN,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,0.265306,4.828724,0.571429,2.161998,0.244898,11.632723,0.204082,6.948946,0.244898,NaN,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,0.540816,7.971851,0.785714,7.771416,0.846939,20.418826,0.724490,13.136821,0.683673,NaN,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,0.632653,1.314494,0.163265,0.356535,0.061224,15.236817,0.326531,7.446478,0.285714,NaN,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,0.326531,12.070547,0.877551,6.108353,0.755102,16.132555,0.448980,7.073559,0.265306,NaN,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,31.362177,0.734694,1.347060,0.183673,0.339561,0.040816,22.086128,0.795918,1.472225,0.020408,NaN,52.102317,1.745207,2.434286,NaN,NaN


**Calculating Robust Value (RV) score**

In [89]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,MMM,149.779999,20.802778,0.285714,18.596970,0.938776,3.242715,0.387755,15.713103,0.387755,9.003108,0.367347,0.473469,145.991219,2.310243,3.091431,NaN,NaN
1,AOS,68.125000,18.976324,0.224490,5.171563,0.591837,2.518687,0.326531,25.644411,0.857143,13.604165,0.714286,0.542857,70.701712,0.778764,1.100714,NaN,NaN
2,ABT,132.755005,16.649311,0.163265,4.567343,0.551020,5.357355,0.632653,20.821846,0.775510,9.822580,0.428571,0.510204,128.589865,5.592050,2.605003,NaN,NaN
3,ABBV,232.729996,110.828580,0.979592,-2237.884500,0.020408,7.048910,0.775510,16.801678,0.469388,11.350715,0.510204,0.551020,198.149631,6.746586,5.626431,NaN,NaN
4,GRMN,252.804993,31.172010,0.693878,5.990781,0.653061,7.200601,0.795918,24.179172,0.836735,11.794173,0.571429,0.710204,216.291986,0.703100,4.348571,NaN,NaN
5,ACN,242.360001,19.932100,0.265306,4.828724,0.571429,2.161998,0.244898,11.632723,0.204082,6.948946,0.244898,0.306122,291.132149,6.339629,6.902144,NaN,NaN
6,GOOG,239.070007,25.509071,0.540816,7.971851,0.785714,7.771416,0.846939,20.418826,0.724490,13.136821,0.683673,0.716327,195.125137,18.886207,5.307857,NaN,NaN
7,ADM,61.439999,26.706522,0.632653,1.314494,0.163265,0.356535,0.061224,15.236817,0.326531,7.446478,0.285714,0.293878,55.529822,3.815986,1.847857,NaN,NaN
8,ADBE,338.265015,21.075703,0.326531,12.070547,0.877551,6.108353,0.755102,16.132555,0.448980,7.073559,0.265306,0.534694,394.700578,4.207236,8.812149,NaN,NaN
9,AAP,49.459999,31.362177,0.734694,1.347060,0.183673,0.339561,0.040816,22.086128,0.795918,1.472225,0.020408,0.355102,52.102317,1.745207,2.434286,NaN,NaN


**Removing Glamour stocks**


In [90]:
rv_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
rv_dataframe = rv_dataframe[rv_dataframe['Price-to-Earnings Ratio'] > 0]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis=1, inplace = True)

rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,APA,22.590000,7.813148,0.020408,1.371894,0.204082,0.824033,0.142857,2.396211,0.020408,2.156626,0.040816,0.085714,21.616980,7.134021,1.025714,NaN,NaN
1,ALL,206.875000,9.726588,0.040816,2.478238,0.428571,0.822494,0.122449,7.343794,0.102041,3.347641,0.122449,0.163265,198.236525,1.251000,3.466428,NaN,NaN
2,AES,14.335000,11.023077,0.061224,3.028318,0.469388,0.848044,0.163265,15.087635,0.306122,22.794481,0.918367,0.383673,12.931379,14.491164,0.580714,NaN,NaN
3,MO,66.745003,12.907157,0.081633,-34.450180,0.040816,5.533300,0.653061,10.648647,0.163265,9.213146,0.387755,0.265306,59.795568,7.420650,0.983571,NaN,NaN
4,AAL,11.555000,13.744048,0.102041,-1.968457,0.081633,0.140419,0.020408,7.105562,0.081633,2.728348,0.061224,0.069388,12.370280,84.492679,0.412857,NaN,NaN
5,AMP,480.260010,14.966033,0.122449,7.454907,0.734694,2.490367,0.306122,18.891573,0.622449,4.057688,0.142857,0.385714,501.794691,0.481893,8.849282,NaN,NaN
6,AIG,81.805000,15.283177,0.142857,1.102846,0.122449,1.653458,0.224490,5.295319,0.040816,4.770730,0.163265,0.138776,79.689852,4.123971,1.393572,NaN,NaN
7,ABT,132.755005,16.649311,0.163265,4.567343,0.551020,5.357355,0.632653,20.821846,0.775510,9.822580,0.428571,0.510204,128.589865,5.592050,2.605003,NaN,NaN
8,AEP,116.680000,17.111437,0.183673,2.088441,0.306122,3.019860,0.367347,12.808886,0.224490,10.896533,0.489796,0.314286,105.429506,3.589093,2.054285,NaN,NaN
9,APTV,81.625000,18.472850,0.204082,1.834873,0.285714,0.898257,0.183673,8.016517,0.122449,6.632249,0.224490,0.204082,71.436495,1.850086,1.710715,NaN,NaN


**Removing downtreding stocks (<EMA200)**


In [91]:

rv_dataframe = rv_dataframe[rv_dataframe['Price'] > rv_dataframe['EMA200']]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis=1, inplace = True)

rv_dataframe

C:\Users\serge\AppData\Local\Temp\ipykernel_6208\338777760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.drop('index', axis=1, inplace = True)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,APA,22.590000,7.813148,0.020408,1.371894,0.204082,0.824033,0.142857,2.396211,0.020408,2.156626,0.040816,0.085714,21.616980,7.134021,1.025714,NaN,NaN
1,ALL,206.875000,9.726588,0.040816,2.478238,0.428571,0.822494,0.122449,7.343794,0.102041,3.347641,0.122449,0.163265,198.236525,1.251000,3.466428,NaN,NaN
2,AES,14.335000,11.023077,0.061224,3.028318,0.469388,0.848044,0.163265,15.087635,0.306122,22.794481,0.918367,0.383673,12.931379,14.491164,0.580714,NaN,NaN
3,MO,66.745003,12.907157,0.081633,-34.450180,0.040816,5.533300,0.653061,10.648647,0.163265,9.213146,0.387755,0.265306,59.795568,7.420650,0.983571,NaN,NaN
4,AIG,81.805000,15.283177,0.142857,1.102846,0.122449,1.653458,0.224490,5.295319,0.040816,4.770730,0.163265,0.138776,79.689852,4.123971,1.393572,NaN,NaN
5,ABT,132.755005,16.649311,0.163265,4.567343,0.551020,5.357355,0.632653,20.821846,0.775510,9.822580,0.428571,0.510204,128.589865,5.592050,2.605003,NaN,NaN
6,AEP,116.680000,17.111437,0.183673,2.088441,0.306122,3.019860,0.367347,12.808886,0.224490,10.896533,0.489796,0.314286,105.429506,3.589093,2.054285,NaN,NaN
7,APTV,81.625000,18.472850,0.204082,1.834873,0.285714,0.898257,0.183673,8.016517,0.122449,6.632249,0.224490,0.204082,71.436495,1.850086,1.710715,NaN,NaN
8,MMM,149.779999,20.802778,0.285714,18.596970,0.938776,3.242715,0.387755,15.713103,0.387755,9.003108,0.367347,0.473469,145.991219,2.310243,3.091431,NaN,NaN
9,LNT,68.080002,21.012346,0.306122,2.448481,0.387755,4.223023,0.530612,15.585555,0.367347,14.475722,0.775510,0.473469,62.535770,1.817207,0.950716,NaN,NaN


Selecting the 5 Best Value Stocks  by sorting the DataFrame on the RV Score column and dropping all but the top 5 entries.

In [92]:
rv_dataframe.sort_values(by='RV Score', ascending=False, na_position='last', inplace=True)
rv_dataframe = rv_dataframe.head(5).reset_index(drop=True)

rv_dataframe

C:\Users\serge\AppData\Local\Temp\ipykernel_6208\275880866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values(by='RV Score', ascending=False, na_position='last', inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,ANET,156.199997,61.160790,0.938776,17.980171,0.918367,24.653763,1.000000,54.752886,0.979592,37.206663,0.979592,0.963265,112.714109,7.569221,6.017429,NaN,NaN
1,AAPL,248.839996,37.774315,0.857143,56.094555,1.000000,9.026988,0.918367,26.933565,0.897959,20.008381,0.897959,0.914286,223.228502,48.506671,4.352861,NaN,NaN
2,AMD,219.845001,130.767870,1.000000,5.972271,0.632653,12.044680,0.959184,68.232858,1.000000,24.911414,0.959184,0.910204,144.419629,72.692229,11.934999,NaN,NaN
3,APH,122.849998,48.940240,0.897959,13.011333,0.897959,7.969919,0.877551,30.487761,0.938776,23.987345,0.938776,0.910204,93.902597,7.618000,2.924285,NaN,NaN
4,ADI,230.095001,58.540710,0.918367,3.320512,0.489796,10.896695,0.938776,26.311178,0.877551,19.569015,0.857143,0.816327,227.620424,2.993557,5.250713,NaN,NaN


**Backtesting module**


**Calculating Cumulative return Column** 

In [93]:
# assumes: rv_dataframe exists with a 'Ticker' column, and `end` is defined (e.g., end=pd.Timestamp.today().strftime('%Y-%m-%d'))

for i in range(len(rv_dataframe)):
    Symbol = str(rv_dataframe.iloc[i]['Ticker']).strip()   # take ticker from the current row and normalize to string
    if not Symbol:                                         # skip if empty ticker
        continue

    # ------------------ 1) Download & normalize to plain OHLCV columns ------------------
    try:
        df = yf.download(
            Symbol.replace('.', '-'),                      # Yahoo-style ticker (e.g., BRK.B -> BRK-B)
            start="1999-05-01", end=end,                   # long history; end is provided upstream
            auto_adjust=False, progress=False,             # raw OHLCV; no auto-adjust; quiet download
            group_by="column"                              # yfinance may return MultiIndex columns; we’ll handle below
        )
        if df is None or df.empty:                         # if nothing came back, skip this ticker
            print(f"[skip] no data for {Symbol}")
            continue

        # Flatten to single-level columns with the selected ticker if MultiIndex
        if isinstance(df.columns, pd.MultiIndex):          # handle multi-level columns (common in yf when multiple tickers or group_by)
            sym_key = Symbol.replace('.', '-')
            lv0 = df.columns.get_level_values(0).astype(str)
            lv1 = df.columns.get_level_values(1).astype(str)
            fields = {"Open","High","Low","Close","Volume","Adj Close"}

            # Case A: level 0 fields, level 1 ticker  -> pick the current ticker’s slice
            if fields.issubset(set(lv0)) and sym_key in set(lv1):
                df = df.xs(sym_key, axis=1, level=1, drop_level=True)
            # Case B: level 0 ticker, level 1 fields  -> same idea, other orientation
            elif sym_key in set(lv0) and fields.issubset(set(lv1)):
                df = df.xs(sym_key, axis=1, level=0, drop_level=True)
            else:
                print(f"[skip] {Symbol}: unexpected columns {list(df.columns)[:6]}")  # diagnostic: structure not recognized
                continue

        # Map columns case-insensitively
        have = {c.lower().strip(): c for c in df.columns}  # build a lowercased map to original column names
        needed = ['open', 'high', 'low', 'close', 'volume']
        if any(col not in have for col in needed):         # ensure core OHLCV present
            print(f"[skip] {Symbol}: missing {needed} in {list(df.columns)}")
            continue

        # Build standard frame with fallback for Adj Close
        df_std = pd.DataFrame({                            # construct a clean single-level OHLCV(+Adj Close) frame
            'Open'  : pd.to_numeric(df[have['open']],  errors='coerce'),
            'High'  : pd.to_numeric(df[have['high']],  errors='coerce'),
            'Low'   : pd.to_numeric(df[have['low']],   errors='coerce'),
            'Close' : pd.to_numeric(df[have['close']], errors='coerce'),
            'Volume': pd.to_numeric(df[have['volume']], errors='coerce')
        })
        if 'adj close' in have:                            # prefer adjusted close when available
            df_std['Adj Close'] = pd.to_numeric(df[have['adj close']], errors='coerce')
        else:
            df_std['Adj Close'] = df_std['Close']          # fallback to Close if Adj Close is missing

        # Clean for yahoofeed (no blanks; Volume must be int)
        df_std = df_std.dropna(subset=['Open','High','Low','Close','Adj Close'])  # drop rows with missing core prices
        df_std['Volume'] = df_std['Volume'].fillna(0).astype('int64')            # volume required as integer

        # Save EXACT Yahoo schema to Symbol.csv
        out = df_std.reset_index()                          # bring DatetimeIndex out as first column
        if 'Date' not in out.columns:                       # older pandas may name the index column differently
            out = out.rename(columns={out.columns[0]: 'Date'})
        out['Date'] = pd.to_datetime(out['Date']).dt.strftime('%Y-%m-%d')        # ensure YYYY-MM-DD string format
        out[['Date','Open','High','Low','Close','Volume','Adj Close']].to_csv('Symbol.csv', index=False)  # write CSV

    except Exception as e:
        print(f"{Symbol}: data/csv error -> {e}")           # any data/IO issue: log and skip
        continue

    # ------------------ 2) Month-end trading calendar ------------------
    try:
        nyse = market_calendar.get_calendar('NYSE')         # get NYSE calendar
        sched = nyse.schedule(start_date='2000-01-01', end_date=end)  # full schedule
        if sched.empty:
            print(f"{Symbol}: empty calendar")
            continue
        sched = sched.groupby(sched.index.strftime('%Y-%m')).tail(1)  # last trading day of each month
        sched['date'] = pd.to_datetime(sched['market_open']).dt.date  # extract dates from timestamps
        last_days_of_month = {d.isoformat() for d in sched['date'].tolist()}  # set for fast membership checks
    except Exception as e:
        print(f"{Symbol}: calendar error -> {e}")           # calendar errors: log and skip
        continue

    # ------------------ 3) Strategy (unchanged logic) ------------------
    class MovingAverageStrategy(strategy.BacktestingStrategy):
        def __init__(self, feed, instrument):
            super(MovingAverageStrategy, self).__init__(feed)
            self.instrument = instrument
            self.position = None
            self.ma = ma.SMA(feed[instrument].getAdjCloseDataSeries(), 200)  # 200-day SMA on adjusted closes
            self.setUseAdjustedValues(True)                                   # use adjusted values for trading logic

        def onEnterOk(self, position):
            self.position = position                                         # mark position when entry order fills

        def onExitOk(self, position):
            self.position = None                                             # clear position when exit order fills

        def onBars(self, bars):
            if self.ma[-1] is None:                                          # if SMA not ready yet, skip
                return
            bar = bars[self.instrument]
            close = bar.getAdjClose()
            date = bar.getDateTime().date().isoformat()

            if date in last_days_of_month:                                    # only act on last trading day of month
                if self.position is None and close > self.ma[-1]:             # entry rule: price above SMA at month-end
                    cash = self.getBroker().getCash() * 0.98                  # invest ~98% of available cash
                    qty = cash / close
                    self.position = self.enterLong(self.instrument, qty)      # enter long
                elif self.position is not None and close < self.ma[-1]:       # exit rule: price below SMA at month-end
                    self.position.exitMarket()

    # ------------------ 4) Run backtest & record analyzers ------------------
    try:
        feed = yahoofeed.Feed()                                             # pyalgotrade Yahoo-format feed
        feed.addBarsFromCSV('Symbol', 'Symbol.csv')                         # instrument name 'Symbol' -> CSV file
        strat = MovingAverageStrategy(feed, 'Symbol')                       # instantiate strategy

        ret_an = returns.Returns()                                          # cumulative returns analyzer
        dd_an  = drawdown.DrawDown()                                        # drawdown analyzer
        strat.attachAnalyzer(ret_an)
        strat.attachAnalyzer(dd_an)

        strat.run()                                                         # execute backtest

        cum_series = ret_an.getCumulativeReturns()                          # series of cumulative returns
        cum_ret_pct = float(cum_series[-1]) * 100.0 if len(cum_series) > 0 and cum_series[-1] is not None else np.nan

        max_dd = dd_an.getMaxDrawDown()                                     # max drawdown as a fraction (negative)
        max_dd_pct = float(max_dd) * 100.0 if max_dd is not None else np.nan

        rv_dataframe.loc[i, 'Cumulative Return'] = cum_ret_pct              # write results back to your main table
        rv_dataframe.loc[i, 'Max Drawdown'] = max_dd_pct

    except Exception as e:
        print(f"{Symbol}: backtest error -> {e}")                           # keep processing others if one fails
        # leave NaNs for this symbol and continue

rv_dataframe                                                                # display final DataFrame with new columns


2019-07-01 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill Symbol order [11] for 88672 share/s
2000-03-01 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill Symbol order [1] for 1139184 share/s
2011-01-03 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill Symbol order [15] for 351233 share/s


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,ANET,156.199997,61.160790,0.938776,17.980171,0.918367,24.653763,1.000000,54.752886,0.979592,37.206663,0.979592,0.963265,112.714109,7.569221,6.017429,46.819050,50.812258
1,AAPL,248.839996,37.774315,0.857143,56.094555,1.000000,9.026988,0.918367,26.933565,0.897959,20.008381,0.897959,0.914286,223.228502,48.506671,4.352861,0.000000,0.000000
2,AMD,219.845001,130.767870,1.000000,5.972271,0.632653,12.044680,0.959184,68.232858,1.000000,24.911414,0.959184,0.910204,144.419629,72.692229,11.934999,193.172347,69.386838
3,APH,122.849998,48.940240,0.897959,13.011333,0.897959,7.969919,0.877551,30.487761,0.938776,23.987345,0.938776,0.910204,93.902597,7.618000,2.924285,1301.818800,63.437457
4,ADI,230.095001,58.540710,0.918367,3.320512,0.489796,10.896695,0.938776,26.311178,0.877551,19.569015,0.857143,0.816327,227.620424,2.993557,5.250713,8.215136,74.638133


Removing low return stocks Cum Return<100% and stocks with high volatility > 100%


In [181]:
rv_dataframe.sort_values('Cumulative Return', inplace = True)
rv_dataframe = rv_dataframe[rv_dataframe['Cumulative Return'] > 100]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis=1, inplace = True)

rv_dataframe.sort_values('Max Drawdown', inplace = True)
rv_dataframe = rv_dataframe[rv_dataframe['Max Drawdown'] < 100]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis=1, inplace = True)

rv_dataframe

C:\Users\serge\AppData\Local\Temp\ipykernel_10492\2198263539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.drop('index', axis=1, inplace = True)
C:\Users\serge\AppData\Local\Temp\ipykernel_10492\2198263539.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('Max Drawdown', inplace = True)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,EMA200,AVG_VOL,ATR,Cumulative Return,Max Drawdown
0,APH,126.250000,50.099205,0.897959,13.372524,0.897959,8.191162,0.877551,30.380110,0.938776,23.902646,0.938776,0.910204,92.633283,9.335086,3.082142,1257.082642,63.437498
1,AMD,232.889999,139.455100,1.000000,6.331113,0.673469,12.768380,0.979592,69.019249,1.000000,25.198521,0.959184,0.922449,141.291978,43.491443,5.426428,193.172347,69.386838


Exporting selected stocks in csv format

In [96]:
rv_dataframe.to_csv("Quantitative Value Stocks.csv")